In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/home/onur/WORK/DS/repos/LLM/datatalks_llm/src/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [4]:
from openai import OpenAI

openai_client = OpenAI()

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    print(answer)
    search_results_str = ''
    for doc in search_results:
        search_results_str += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    print(f'sources: {search_results_str}')
    # return answer

In [13]:
rag('how do I run kafka?')

To run Kafka, you need to execute the appropriate Java commands or Python commands based on your setup. 

For Java Kafka, navigate to the project directory and run the following command in the terminal:

```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

If you're using Python, you should create a virtual environment and install the necessary dependencies as follows:

1. Create the virtual environment:
   ```
   python -m venv env
   ```

2. Activate the virtual environment:
   ```
   source env/bin/activate
   ```

3. Install the requirements:
   ```
   pip install -r ../requirements.txt
   ```

Remember to run the activation command each time you need to use the virtual environment. For Windows, the activation command will be:
```
env\Scripts\activate
```

Make sure that all necessary Docker images are up and running if your setup involves Docker.
sources: section: Module 6: streaming with kafka
question: Java Kafka: How to run pr

In [14]:
rag('the course has already started, can I still enroll?')

Yes, you can still enroll in the course even after it has started. You are eligible to submit homework assignments, but be aware that there will be deadlines for the final projects, so it's advisable not to leave everything until the last minute.
sources: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: Genera

## RAG with Vector Search

In [15]:
from qdrant_client import QdrantClient, models

In [16]:
qd_client = QdrantClient("http://localhost:6333")

In [17]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [18]:
collection_name = "zoomcamp-faq"

In [19]:
qd_client.delete_collection(collection_name=collection_name)

False

In [20]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [21]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [23]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [25]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    print(answer)
    search_results_str = ''
    for doc in search_results:
        search_results_str += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    print(f'sources: {search_results_str}')
    # return answer

In [26]:
rag('how do I run kafka?')

vector_search is used
To run Kafka, specifically to execute the producer, consumer, or kstreams Java scripts, you should follow these steps in your project directory:

1. Open your terminal.
2. Run the following command, replacing `<jar_name>` with the name of your jar file:
   ```
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```
   to run the producer, or change `JsonProducer.java` to `JsonConsumer.java` or other scripts as needed.

Make sure that your Kafka broker is also running by checking with `docker ps`, and if necessary, start it with `docker compose up -d` in the docker compose yaml file folder.
sources: section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Java Kafka: When run

In [27]:
rag('the course has already started, can I still enroll?')

vector_search is used
Yes, you can still enroll in the course after it has started. Even if you don't register, you're eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute.
sources: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start worki